In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

url = 'https://en.wikipedia.org/wiki/List_of_professional_wrestling_matches_rated_5_or_more_stars_by_Dave_Meltzer#Wrestlers_with_the_most_5-or-higher-star_matches'

req = requests.get(url)
print(f"Request terminated with status code {req.status_code}")
print(f"Response encoded with {req.encoding}")
# as before, we can add the HTML to our soup
wiki_soup = BeautifulSoup(req.text, 'html.parser')

Request terminated with status code 200
Response encoded with UTF-8


In [51]:
tables = wiki_soup('table') # your code goes here
# and here we need to look the table titles to find id of the table that interests us

matches_df = pd.read_html(str(tables[0]))[0]
for i in range(1,5):
    matches_df = pd.concat([matches_df, pd.read_html(str(tables[i]))[0]])

# These columns are fluff that are only helpful in wikipedi
matches_df = matches_df.drop(['Unnamed: 0', 'Ref.'], axis=1)
matches_df = matches_df.reset_index(drop=True)

# Reformatting the rating to be a series of floats
matches_df['Rating'] = matches_df['Rating'].astype(str).map(lambda x: x.rstrip('[]abcd')).astype(float)

# Manually fixing one of the entries, because this is honestly the easiest way to proceed
matches_df['Match'][0] = matches_df['Match'][0][:-3]

<ipython-input-51-202cdd16a680>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches_df['Match'][0] = matches_df['Match'][0][:-3]


In [251]:
# Splitting who fought who
temp_split = matches_df['Match'].str.split('vs.', expand=True)

# Splitting each person in teams
temp = temp_split.apply(lambda x: x.str.replace('and', ',')).apply(lambda x: x.str.split(','))

def clean_up_list(x):
    for col in x:
        if col is not None:
            if ' ' in col:
                # At some point empty characters are created, beats me why
                col.remove(' ')
            for i in range(len(col)):
                elem = col[i]
                # This is to deal with tag team names, just removing them and only leave the wrestler's name
                elem = re.sub(r'^.*?\(', '', elem)
                elem = elem.replace(')', '')
                elem = elem.strip()
                col[i] = elem
    return x

temp.apply(lambda row: clean_up_list(row))

new_column_names = []
for elem in temp.columns:
    new_column_names.append('Participant ' + str(elem+1))

temp.columns = new_column_names

matches_df = pd.concat([matches_df.drop('Match', axis=1), temp], axis=1)

In [252]:
matches_df

,Date,Promotion,Event,Rating,Participant 1,Participant 2,Participant 3,Participant 4,Participant 5,Participant 6
0,"April 7, 1982",CWF,Miami Beach show,5.00,[Ric Flair],[Butch Reed],None,None,None,None
1,"April 21, 1983",NJPW,Big Fight Series IINight 19,5.00,[Dynamite Kid],[Tiger Mask],None,None,None,None
2,"December 5, 1984",UWF,Year-End SpecialDay 10,5.00,[Kazuo Yamazaki],[Nobuhiko Takada],None,None,None,None
3,"December 8, 1984",AJPW,Real World Tag LeagueNight 15,5.00,"[Bruiser Brody, Stan Hansen]","[Dory Funk Jr., Terry Funk]",None,None,None,None
4,"March 9, 1985",AJPW,85 Gekitoh! Exciting WarsNight 14,5.00,[Kuniaki Kobayashi],[Tiger Mask],None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
191,"November 13, 2021",AEW,Full Gear,5.50,[Adam Page],[Kenny Omega],None,None,None,None
192,"December 15, 2021",AEW,Winter is Coming,5.00,[Adam Page],[Bryan Danielson],None,None,None,None
193,"January 5, 2022",NJPW,Wrestle Kingdom 16Night 2,5.75,[Kazuchika Okada],[Will Ospreay],None,None,None,None
194,"January 26, 2022",AEW,Beach Break,5.00,[Cody Rhodes],[Sammy Guevara],None,None,None,None
